In [1]:
import json

kb_data = {
    "plans": {
        "Basic": {
            "price": "$29/month",
            "videos": "10 videos/month",
            "resolution": "720p"
        },
        "Pro": {
            "price": "$79/month",
            "videos": "Unlimited videos",
            "resolution": "4K resolution",
            "features": ["AI captions"]
        }
    },
    "policies": {
        "refund": "No refunds after 7 days",
        "support": "24/7 support available only on Pro plan",
        "support_email": "support@autostream.com"
    }
}

with open("autostream_kb.json", "w") as f:
    json.dump(kb_data, f, indent=4)

print("Knowledge base created successfully!")


Knowledge base created successfully!


In [2]:
with open("autostream_kb.json", "r") as f:
    kb = json.load(f)

print("Knowledge base loaded!")


Knowledge base loaded!


In [3]:
def mock_lead_capture(name, email, platform):
    return f"Lead captured successfully: {name}, {email}, {platform}"


In [4]:
conversation_state = {
    "memory": [],  # store last 5–6 messages
    "lead_info": {"name": None, "email": None, "platform": None}
}

def update_memory(user_input, agent_response):
    conversation_state["memory"].append({"user": user_input, "agent": agent_response})
    if len(conversation_state["memory"]) > 6:
        conversation_state["memory"].pop(0)


In [5]:
def identify_intent(user_input):
    text = user_input.lower()
    greetings = ["hi", "hello", "hey", "good morning", "good evening"]
    polite = ["sorry", "thank you", "thanks", "bye", "goodbye", "ok", "okie"]
    high_intent_keywords = ["sign up", "try", "pro plan", "subscribe", "buy"]
    product_keywords = ["price", "pricing", "cost", "features", "plans"]

    if any(word in text for word in greetings):
        return "greeting"
    elif any(word in text for word in polite):
        return "polite"
    elif any(word in text for word in high_intent_keywords):
        return "high_intent"
    elif any(word in text for word in product_keywords):
        return "product_inquiry"
    elif "support" in text or "human" in text or "help" in text:
        return "support"
    else:
        return "unknown"


In [6]:
def retrieve_answer(user_input):
    text = user_input.lower()
    
    # Product plans
    if "price" in text or "plan" in text or "features" in text:
        answer = "Here are our plans:\n"
        for plan, details in kb["plans"].items():
            answer += f"\n{plan} Plan:\n"
            for k, v in details.items():
                if k == "features":
                    answer += f"{k.capitalize()}: {', '.join(v)}\n"
                else:
                    answer += f"{k.capitalize()}: {v}\n"
        return answer

    # Company policies
    elif "refund" in text or "support" in text:
        policies = kb["policies"]
        return (f"Refund Policy: {policies['refund']}\n"
                f"Support Policy: {policies['support']}\n"
                f"Support Email: {policies['support_email']}")
    
    else:
        return (f"I’m not sure about that. You can contact our support assistant at "
                f"{kb['policies']['support_email']}")


In [7]:
def run_agent(user_input):
    lead_info = conversation_state["lead_info"]
    intent = identify_intent(user_input)
    response = ""
    text = user_input.lower()

    # Greeting
    if intent == "greeting":
        response = "Hello! 👋 I’m your AutoStream assistant. How can I help you today?"

    # Polite messages
    elif intent == "polite":
        if "thank" in text:
            response = "You’re welcome! 😊"
        elif "sorry" in text:
            response = "No worries! How can I assist you further?"
        elif "bye" in text or "goodbye" in text:
            response = "Goodbye! 👋 Have a great day!"
        else:
            response = "Okay! How can I help you next?"

    # Support request
    elif intent == "support":
        response = f"Connecting you to a support assistant. You can also email them at {kb['policies']['support_email']}"

    # Product inquiry
    elif intent == "product_inquiry":
        response = retrieve_answer(user_input)

    # High-intent lead capture
    elif intent == "high_intent":
        if not lead_info["name"]:
            response = "Awesome! Can I get your name?"
        elif not lead_info["email"]:
            response = "Thanks! Please provide your email."
        elif not lead_info["platform"]:
            response = "Which platform do you create content on? (YouTube, Instagram, etc.)"
        else:
            response = mock_lead_capture(lead_info["name"], lead_info["email"], lead_info["platform"])
            conversation_state["lead_info"] = {"name": None, "email": None, "platform": None}

    # Unknown queries
    else:
        response = retrieve_answer(user_input)

    # Update memory
    update_memory(user_input, response)
    return response


In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Agent: Goodbye! 👋")
        break
    
    lead_info = conversation_state["lead_info"]
    agent_response = run_agent(user_input)
    print("Agent:", agent_response)

    # Stepwise lead info collection
    if "Can I get your name" in agent_response:
        lead_info["name"] = input("Enter Name: ")
    elif "please provide your email" in agent_response:
        lead_info["email"] = input("Enter Email: ")
    elif "Which platform" in agent_response:
        lead_info["platform"] = input("Enter Platform: ")


You:  Hi


Agent: Hello! 👋 I’m your AutoStream assistant. How can I help you today?
